# Симплициальные многогранники и внешние формы

## Порядок сдачи
Срок сдачи задания -- **1 апреля**. 2 апреля работы будут собраны и уйдут на проверку. Сдача после дедлайна возможна в индивидуальном порядке за половинный балл.

Можете объединяться в команды по 2-3 человека для совместного написания программ:
для этого нужно пригласить соответствующего студента в коллабораторы и указать в комментариях к коду, кто реализовал какое задание.
Можете реализовывать задания по своему выбору. 
Максимальная сумма, присуждаемая одному участнику -- **30** баллов.
Если команда (из 1 или более человека) корректно реализовала все методы -- дополнительно присуждается по **10** баллов каждому.

Будьте готовы к тому, что мы попросим некоторых из вас объяснить, что делает программа и как работает код. Использование чужого кода наказуемо аннулированием баллов у всех сторон подобного инцидента.
По всем вопросам пишите в чат в правом верхнем углу - мы увидим оповещение и ответим. Вопросы общего характера можно также задавать в telegram-чате.

## Соглашения

Мы задаём ориентируемое симплициальное многообразие X без края с помощью класса Variety. Оно обладает X.n вершинами, пронумерованными числами от 0 до n-1.
Рёбра задаются парами вершин, (полу)рёбра -- парами вершин с учётом их порядка, а грани задаются тройками вершин с учётом ориентации, то есть направления обхода вершин.
Основным полем объекта X является X.faces -- массив ориентированных граней.
- *0-форма* (функция на вершинах) является функцией, принимающей номер вершины и возвращающей значение.
- *1-форма* (функция на рёбрах) является функцией, принимающей пару номеров вершин и возвращающей значение. Если пара вершин не задаёт ребро, то функция вольна возвращать произвольное значение. 
- *2-форма* (функция на гранях) является функцией, принимающей тройку номеров вершин и возвращающей значение. Если тройка вершин не задаёт грань, то функция вольна возвращать произвольное значение. 

## Задания

1. *(5 баллов)* Напишите метод **check**(), проверяющий, задаёт ли faces (массив троек номеров вершин) симплициальное многообразие (ориентируемое, без края), и возвращающий соответствующее булево значение. Вам нужно проверить, что окрестность каждой вершины и окрестность каждого ребра гомеоморфны плоскости. *Указание:* удобнее проверять на полурёбрах.
2. *(5 баллов)* Напишите метод **Euler**(), возвращающий эйлерову характеристику симплициального многообразия.
3. *(5 баллов)* Напишите метод **d_0**(form), принимающий 0-форму и возвращающий её дифференциал -- 1-форму.
4. *(5 баллов)* Напишите метод **check_form**(k, form), принимающий на вход k-форму и проверяющий её корректность, где k = 1 или 2.
5. *(5 баллов)* Напишите метод **d_1**(form), принимающий 1-форму и возвращающий её дифференциал -- 2-форму.
6. Напишите метод **wedge**(k1,k2, form1, form2), принимающий k1-форму и k2-форму и возвращающий их внешнее произведение, для (k1,k2)=
  - *(5 баллов)* (0,0).
  - *(5 баллов)* (0,1) и (1,0).
  - *(10 баллов)* (0,2) и (2,0).
  - *(15 баллов)* (1,1).


In [23]:
import numpy as np

In [24]:
class Variety():
    def __init__(self, faces):
        self.faces = faces
        self.vertices = set(i for f in faces for i in f)
        self.n = max(self.vertices) + 1
        self.matrix_adjacency = np.zeros((self.n, self.n))
        assert set(range(self.n)) == self.vertices
        for f in faces:
            assert len(f)==3

    @property
    def num_vertices(self):
        return self.n
    
    def check(self):
        unique_faces = []
        error_flag = 0
        for f in self.faces:
            f_sort = sorted(f)
            if f_sort in unique_faces:
                error_flag = 1
            else:
                unique_faces.append(f_sort)
            
        if error_flag:
            return False
            
        for f in self.faces:
            for i in range(3):
                self.matrix_adjacency[f[i]][f[(i + 1) % 3]] += 1
        
        error_flag = 0
        indicators = self.matrix_adjacency > 1
        for arr in indicators:
            for ind in arr:
                if ind == True:
                    error_flag = 1
        
        if error_flag:
            return False
        
        error_flag = 0
        indicators = self.matrix_adjacency != self.matrix_adjacency.T
        for arr in indicators:
            for ind in arr:
                if ind == True:
                    error_flag = 1
        
        if error_flag:
            return False
        
        for vertex in self.vertices:
            size = 0
            edges = {}
            
            for (x, y, z) in self.faces:
                if vertex in (x, y, z):
                    size += 1
                    if vertex == x:
                        edges[y] = z
                    if vertex == y:
                        edges[z] = x
                    if vertex == z:
                        edges[x] = y
            
            list_edges = list(edges)
            
            size_path = 1
            v_from = list_edges[0]
            v_to = edges[v_from]
            
            while (v_from != v_to):
                v_to = edges[v_to]
                size_path += 1
                
            if size_path != size:
                return False

        return True
    
    def Euler(self):
        return self.n + len(self.faces) - int(np.sum(self.matrix_adjacency) / 2)

    def d_0(self, form):
        def func(a, b):
            return form(b) - form(a)
        return func

    def check_form(self, k, form):
        if k == 1:
            for f in self.faces:
                a = (f[0], f[1])
                b = (f[1], f[2])
                if form(a, b) != -form(b, a):
                    return False
        if k == 2:
            for f in self.faces:
                a = f[0]
                b = f[1]
                c = f[2]
                
                check1 = (form(a, b, c) == form(b, c, a) == form(c, a, b))
                check2 = (form(a, c, b) == form(b, a, c) == form(c, b, a))
                check3 = (form(a, b, c) == -form(a, c, b))
                
                if (not check1) or (not check2) or (not check3):
                    return False
                
        return True

    def d_1(self, form):
        def func(a, b, c):
            return form(a, b) + form(b, c) + form(c, a)
        return func
    
    def wedge(self, k1, k2, f1, f2):
        if k1 == 0 and k2 == 0:
            def func(a):
                return f1(a) * f2(a)
            return func
        
        if k1 == 0 and k2 == 1:
            def func(a, b):
                f1_sum = f1(a) + f1(b)
                return f1_sum * f2(a, b) / 2
            return func
        
        if k1 == 1 and k2 == 0:
            def func(a, b):
                f2_sum = f2(a) + f2(b)
                return f2_sum * f1(a, b) / 2
            return func
        
        if k1 == 0 and k2 == 2:
            def func(a, b, c):
                f1_sum = f1(a) + f1(b) + f1(c)
                return f1_sum * f2(a, b, c) / 3
            return func
        if k1 == 2 and k2 == 0:
            def func(a, b, c):
                f2_sum = f2(a) + f2(b) + f2(c)
                return f2_sum * f1(a, b, c) / 3
            return func
        
        if k1 == 1 and k2 == 1:
            def func(a, b, c):
                sum1 = f1(a, b) * f2(b, c) + f1(b, c) * f2(c, a) + f1(c, a) * f2(a, b)
                sum2 = f1(b, c) * f2(a, b) + f1(c, a) * f2(b, c) + f1(a, b) * f2(c, a)
                return (sum1 - sum2) / 6
            return func

Запустим файл проверки ошибок:

In [25]:
# примеры проверок (сначала запускаете ячейку с классом, потом с проверкой):


sphere = Variety([(3,2,1), (2,3,0), (1,0,3),(0,1,2)])
torus = Variety([
        (1,0,3),
        (1,3,2),
        (2,3,6),
        (3,4,6),
        (4,0,6),
        (1,6,0),
        (2,6,5),
        (1,5,6),
        (2,5,0),
        (3,0,5),
        (5,4,3),
        (1,4,5),
        (1,2,4),
        (2,0,4),
    ])

def test1():
    assert sphere.check() == True
    assert torus.check() == True
    assert Variety([(1,2,3), (2,3,0), (3,0,1),(0,1,2)]).check() == False
    assert Variety([(1,2,3), (2,3,1), (3,0,1),(0,1,2)]).check() == False
    assert Variety([(1,2,0), (1,0,2)]).check() == False
    assert Variety([(3,2,1), (2,3,0), (1,0,3),(0,1,2), (6,5,4), (5,6,0), (4,0,6),(0,4,5)]).check() == False


def test2():
    assert sphere.Euler() == 2
    assert torus.Euler() == 0

def test3():
    assert sphere.d_0(lambda x : x)(1, 2) == 1
    assert torus.d_0(lambda x : x**2)(4, 3) == -7

def test4():
    assert sphere.check_form(2, lambda x,y,z : x+y+z if (x-y)*(y-z)*(z-x)>0 else -(x+y+z)) == True
    assert torus.check_form(1,lambda v,w : 1) == False

def test5():
    assert sphere.d_1(lambda x, y : x-y)(0, 1, 2) == 0
    assert sphere.d_1(lambda x, y : x*y if x<y else -x*y)(1, 2, 3) == 5

def test61():
    assert sphere.wedge(0,0, lambda x: x, lambda x: -x)(2) == -4

def test62():
    assert sphere.wedge(0,1, lambda x: x, lambda x,y: y-x)(2,3) == 5/2

def test63():
    assert sphere.wedge(0,2, lambda x: x, lambda x,y,z:  x+y+z if (x-y)*(y-z)*(z-x)>0 else -(x+y+z) )(0,1,2) == 3

def test64():
    assert sphere.wedge(1,1, lambda x,y: x-y, lambda x,y: y-x)(1,2,3) == 0

tests = [(1, test1),
         (2, test2),
         (3, test3),
         (4, test4),
         (5, test5),
         ("6.1", test61),
         ("6.2", test62),
         ("6.3", test63),
         ("6.4", test64),
        ]

for i, t in tests:
    try:
        t()
    except:
        print('Функция задания {0} не прошла проверку'.format(i))
    else:
        print('Ошибок в задании {} не найдено'.format(i))

Ошибок в задании 1 не найдено
Ошибок в задании 2 не найдено
Ошибок в задании 3 не найдено
Ошибок в задании 4 не найдено
Ошибок в задании 5 не найдено
Ошибок в задании 6.1 не найдено
Ошибок в задании 6.2 не найдено
Ошибок в задании 6.3 не найдено
Ошибок в задании 6.4 не найдено


**Итог:** Ошибок не обнаруженно!